In [1]:
import nltk, json, random, pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from tensorflow.keras.models import load_model

# **CHATBOT - USE IT NOW**

## **¿How can we use our chatbot?**


<font size="3"> We are writting this in Jupyter because it is easier to follow. However you can put this scripts into a .py file as well. For the chatbot to work we need to import the model and the parameters first. Then we define a series of functions that help us answer different questions our user could have. Let's see how it works. <br><br>

In [2]:
#import the model and the inputs
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [3]:
# Inputs pre-processing
# First we tokenize a sentence. 
# Second we put all the words in small letters
# Third we lemmatize each word.

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

#Example
clean_up_sentence('Hi, how are you today?')

['hi', ',', 'how', 'are', 'you', 'today', '?']

In [4]:
# Create an array of zeros and ones that show if the tokens in the sentence are contained in the words list

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

#Example
bow('Hi, how are you today?', words, show_details=True)

found in bag: hi
found in bag: ,
found in bag: ,
found in bag: how
found in bag: are
found in bag: you
found in bag: you
found in bag: you
found in bag: you


array([1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [5]:
#Forecast the intent and its probability to give an answer

def pred_calculation(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

#Example
pred_calculation('can you please lend me some money', model)

1/1 [==============================] - 0s 76ms/step


[{'intent': 'thanks', 'probability': '0.9566978'}]

In [6]:
#Step by step

#1. Calculate the array of the interaction
sentence ='can you please lend me some money'
p = bow(sentence, words,show_details=False)
print('Array')
print(p)
print('-----------------------------------------------------------------------')
#2. Calculate the probability that the array belongs to a class (in this example we have 3 classes)
res = model.predict(np.array([p]))[0]
print('Probabilities of each class')
print(res)
print('-----------------------------------------------------------------------')
#3. Get those classes with a probability higher than the threshold
ERROR_THRESHOLD = 0.20  #lower threshold than in the function
results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
print('Classes with probability higher than threshold and its respective probability')
print(results)
print('-----------------------------------------------------------------------')
#4. Sort the results
results.sort(key=lambda x: x[1], reverse=True)
print('Sorted classes')
print(results)
print('-----------------------------------------------------------------------')
#5. Sort the results
return_list = []
for r in results:
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
print('Final result')
print(return_list)

Array
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1]
-----------------------------------------------------------------------
1/1 [==============================] - 0s 22ms/step
Probabilities of each class
[0.03962048 0.00368172 0.9566978 ]
-----------------------------------------------------------------------
Classes with probability higher than threshold and its respective probability
[[2, 0.9566978]]
-----------------------------------------------------------------------
Sorted classes
[[2, 0.9566978]]
-----------------------------------------------------------------------
Final result
[{'intent': 'thanks', 'probability': '0.9566978'}]


In [7]:
#Get an answer

def get_answer(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

#Example
get_answer(return_list,intents)

'Happy to help!'

In [8]:
#Step by step

#1. Get the class of the interaction
tag =return_list[0]['intent']
print('Class')
print(tag)
print('-----------------------------------------------------------------------')
#2. Get all the classes and interactions from the trained model
list_of_intents = intents['intents']
print('Classes')
print(list_of_intents)
print('-----------------------------------------------------------------------')
#3. Choose an answer
for i in list_of_intents:
    if(i['tag']== tag):
        result = random.choice(i['responses'])
        break
print('Answer')
print(result)

Class
thanks
-----------------------------------------------------------------------
Classes
[{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['Bye! Have a nice day'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you"], 'context': ['']}]
-----------------------------------------------------------------------
Answer
Happy to help!


In [9]:
#Start the chatbot

def initialize_chatbot(msg):
    ints = pred_calculation(msg, model)
    res = get_answer(ints, intents)
    return res

#Example
initialize_chatbot('Bye')

1/1 [==============================] - 0s 16ms/step


'Bye! Have a nice day'

In [10]:
user = ''
print('Welcome! To exit please write Esc"')

while user != 'Esc':
    user = str(input(""))
    res = initialize_chatbot(user)
    print('AI:' + res)

Welcome! To exit please write Esc"
Hello
1/1 [==============================] - 0s 47ms/step
AI:Hi there, how can I help?
See you!
1/1 [==============================] - 0s 65ms/step
AI:Bye! Have a nice day
Esc
1/1 [==============================] - 0s 57ms/step
AI:Hi there, how can I help?


> *What we have to learn to do, we learn by doing*. *Aristotle*

<font size="3">
Follow me on <a href="https://co.linkedin.com/in/andres-gomez-hernandez" target="_blank">Linkedin</a> for topics about quantitative finance, data science and emerging markets.
</font>